# FashionMNISTを使用したCNNの作成

## 必要なライブライのインんストール

In [ ]:
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

import torchvision
import torchvision.transforms as transforms
from torchvision.transforms.functional import normalize

import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

## データセットのダウンロードと前処理

In [ ]:
#画像のデータの水増し
affine = transforms.RandomAffine([-15,15],scale=(0.7,1.3))#回転とリサイズの定義
horizonal_flip = transforms.RandomHorizontalFlip(p = 0.5) #左右反転させる確率
vertical_flip = transforms.RandomVerticalFlip(p = 0.5) #上下反転させる確率
normalize = transforms.Normalize((0.0),(1.0)) #平均値を０に、標準偏差を1に変更
totensor = transforms.ToTensor()#テンソル化

fashion_mnist_data = torchvision.datasets.FashionMNIST(#fashion_mnistのデータを取ってくる
    './fashion-mnist',train=True,download=True,transform=torchvision.transforms.transforms.ToTensor()
)

transform_train = transforms.Compose([affine,horizonal_flip,vertical_flip,totensor,normalize])#学習用データの前処理
transform_test = transforms.Compose([totensor,normalize])#テスト用データの前処理

#今回の使用するデータであるfashion_mnistのダウンロード
fashion_mnist_data_train = torchvision.datasets.FashionMNIST(
    './fashion-mnist',train=True,download=True,transform=transform_train)
fashion_mnist_data_test = torchvision.datasets.FashionMNIST(
    './fashion-mnist',train=False,download=True,transform=transform_test)

#DataLoaderの設定
batch_size = 32
train_loader = DataLoader(fashion_mnist_data_train,batch_size=batch_size,shuffle=True)
test_loader = DataLoader(fashion_mnist_data_test,batch_size=len(fashion_mnist_data_test),shuffle = False)

## モデルの構築

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),#畳み込みレイヤーとして2D畳み込みを適用する
            nn.ELU(),#活性化関数としてELUを適用する
            nn.MaxPool2d(kernel_size=2),# プーリングレイヤーとして2D最大プーリングを適用する
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ELU(),
            nn.MaxPool2d(kernel_size=2),
        )

        self.classifier = nn.Sequential(
            nn.Dropout(),#入力テンソルの一部をランダムにゼロに設定する
            nn.Linear(64 * 7 * 7, 128),#受信データび線形変換を適用する
            nn.ELU(),
            nn.Dropout(),
            nn.Linear(128, 10),
            nn.LogSoftmax(dim=1),#出力層での活性化関数いついてlogsoftmaxを適用する
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)#全ての次元を平坦化する
        x = self.classifier(x)

        return x
net = Net()
net.cuda() #GPU対応
print(net)

## モデルの訓練(学習)

In [ ]:
#損失関数(交差エントロピー関数)
loss_fnc = nn.CrossEntropyLoss()

#最適化アルゴリズム(adam)
optimizer = optim.AdamW(net.parameters())

#損失ログ
record_loss_train = []
record_loss_test = []

#学習
x_test, t_test = iter(test_loader).next()#test_loaderからデータを取り出す
x_test, t_test = x_test.cuda(), t_test.cuda()#gpuに送信
for i in range(20):#20エポック学習
    net.train()#訓練モード
    loss_train = 0
    for j, (x,t) in enumerate(train_loader):#ミニバッチ(x,t)を取り出す
        x, t = x.cuda(), t.cuda()#GPU対応
        y = net(x)
        loss = loss_fnc(y, t)#損失関数
        loss_train += loss.item()#lossからtensorの要素を所得
        optimizer.zero_grad()#勾配の初期化(backeardする前に行う)
        loss.backward()#逆伝播を行う
        optimizer.step()#重みの更新

    loss_train /= j + 1
    record_loss_train.append(loss_train)

    net.eval()#評価モード
    y_test = net(x_test)
    loss_test = loss_fnc(y_test, t_test).item()
    record_loss_test.append(loss_test)

    if i % 1 == 0:
        print("Epoch:", i, "Loss_Train:", loss_train, "Loss_Test:", loss_test)

## 誤差の推移
訓練データ、テストデータで誤差の推移をグラフ表示

In [ ]:
plt.plot(range(len(record_loss_train)), record_loss_train, label="Train")
plt.plot(range(len(record_loss_test)), record_loss_test, label = "Test")
plt.legend()

plt.xlabel("Epochs")
plt.ylabel("Error")
plt.show()

## 正解率の表示

In [ ]:
correct = 0
total = 0
net.eval() #評価モード
for i, (x,t) in enumerate(test_loader):
    x, t = x.cuda(), t.cuda()#GPU対応
    y = net(x)
    correct += (y.argmax(1) == t).sum().item()
    total += len(x)
print("正解率:", str(correct/total*100) + "%")

## モデルの保存

In [ ]:
torch.save(net.state_dict(),"drive/My Drive/Python_learning/fashion_mnist_cnn/model_cnn_fashin_mnist.pth")

## モデルの読み込み

In [ ]:
net_loaded = Net()
net_loaded.load_state_dict(torch.load("drive/My Drive/Python_learning/fashion_mnist_cnn/model_cnn_fashin_mnist.pth",map_location=torch.device("cpu")))#cpu対応
net_loaded.eval() #評価モード